# Knowledge Distillation in MHIST Dataset

In [1]:
import os
import sys
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

plt.style.use('_mpl-gallery')

from Utilities import *

# Load dataset

In [2]:
# Prepare data
path = "mhist_dataset/images"
CSVfile = "mhist_dataset/annotations.csv"

data, file_names = load_mhist_images(path)
X_train, y_train, X_test, y_test = loadMHIST(CSVfile,data)

# Data Augmentation
AugmentedData = []
AugmentedLabel = []
for i in range(len(X_train)):
    if((y_train[i]==(1,0)).all()):
        AugmentedData.append(cv2.rotate(X_train[i], cv2.ROTATE_90_CLOCKWISE))
        AugmentedLabel.append([1,0])
X_train = np.concatenate((X_train, AugmentedData))
y_train = np.concatenate((y_train, AugmentedLabel))
c = list(zip(X_train, y_train))

random.shuffle(c)
X_train, y_train = zip(*c)
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

m  = np.mean(X_train)
s  = np.std(X_train)
X_train = (X_train - m )/ s
X_test = (X_test - m )/ s


Train_Data, Train_Label, Test_Data, Test_Label = dataBatching(X_train, y_train, X_test, y_test)


630 1545


In [3]:
## Load Teacher Model
restNet = getresnetModel()

## Load Student Model for KD
mobileNetKD = getMobileNetModel()

## Load Student Model for from scratch training
mobileNet = getMobileNetModel()

# Train Teacher Model (Resnet)

In [4]:
## intial epochs at learning rate 0.0001
resnet10EpochPerformance = train_and_evaluate(restNet,Train_Data,Test_Data,Train_Label,Test_Label,10, 0.0001)
## Fine tune epochs at learning rate 0.00001
resnet25EpochPerformance = train_and_evaluate(restNet,Train_Data,Test_Data,Train_Label,Test_Label,25, 0.00001)


Epoch 1: Training Accuracy: 76.69 % and Testing Accuracy: 56.05 % Sensitivity: 73.58% Specificity: 30.83 %
Epoch 2: 

# Train Student Mobile Net using KD


In [ ]:
## intial epochs
mobilenetKd10EpochPerformance = train_and_evaluate_mobileNet_using_KD(mobileNetKD,restNet,Train_Data,Test_Data,Train_Label,Test_Label, 0.5,4, 10, 0.001)
## Fine tune epochs
mobilenetKd25EpochPerformance = train_and_evaluate_mobileNet_using_KD(mobileNetKD,restNet,Train_Data,Test_Data,Train_Label,Test_Label, 0.5,4, 25, 0.0001)

# Train another student model from scratch

In [ ]:
## intial epochs
mobilenet10EpochPerformance = train_and_evaluate(mobileNet,Train_Data,Test_Data,Train_Label,Test_Label,10, 0.001)
## Fine tune epochs
mobilenet25EpochPerformance = train_and_evaluate(mobileNet,Train_Data,Test_Data,Train_Label,Test_Label,25, 0.0001)


# Models Testing

In [ ]:
# model testing
teacherPerformance = testModelPerformance(restNet,Test_Data,Test_Label)
studentPerfromance = testModelPerformance(mobileNet,Test_Data,Test_Label)
studentKdPerfromance = testModelPerformance(mobileNetKD,Test_Data,Test_Label)

# Draw Performance Measures

In [ ]:
# combine data
resnetTraining = np.squeeze(np.asarray([resnet10EpochPerformance[0] + resnet25EpochPerformance[0]]))
resnetTesting = np.squeeze(np.asarray([resnet10EpochPerformance[1] + resnet25EpochPerformance[1]]))
mobilenetKdTraining = np.squeeze(np.asarray([mobilenetKd10EpochPerformance[0] + mobilenetKd25EpochPerformance[0]]))
mobilenetKdTesting = np.squeeze(np.asarray([mobilenetKd10EpochPerformance[1] + mobilenetKd25EpochPerformance[1]]))
mobilenetTraining = np.squeeze(np.asarray([mobilenet10EpochPerformance[0] + mobilenet25EpochPerformance[0]]))
mobilenetTesting = np.squeeze(np.asarray([mobilenet10EpochPerformance[1] + mobilenet25EpochPerformance[1]]))

In [ ]:
f, ax = plt.subplots(2,1,figsize=(16, 10))
ax[0].plot(resnetTraining[:,0], label = 'ResNet training Accuracy' )
ax[0].plot(resnetTraining[:,1], label = 'ResNet training Sensitivity' )
ax[0].plot(resnetTraining[:,2], label = 'ResNet training Specificity' )
ax[0].plot(mobilenetKdTraining[:,0], label = 'MobileNet using KD training Accuracy' )
ax[0].plot(mobilenetKdTraining[:,1], label = 'MobileNet using KD training Sensitivity' )
ax[0].plot(mobilenetKdTraining[:,2], label = 'MobileNet using KD training Specificity' )
ax[0].plot(mobilenetTraining[:,0], label = 'MobileNet training Accuracy' )
ax[0].plot(mobilenetTraining[:,1], label = 'MobileNet training Sensitivity' )
ax[0].plot(mobilenetTraining[:,2], label = 'MobileNet training Specificity' )
ax[0].set_xlabel('Epochs', fontsize = 16)
ax[0].set_ylabel('Percentage (%)', fontsize = 16)
ax[0].set_title(' Training Performance Across Epochs', fontsize = 20)
ax[0].legend()
ax[1].plot(resnetTesting[:,0], label = 'ResNet training Accuracy' )
ax[1].plot(resnetTesting[:,1], label = 'ResNet training Sensitivity' )
ax[1].plot(resnetTesting[:,2], label = 'ResNet training Specificity' )
ax[1].plot(mobilenetKdTesting[:,0], label = 'MobileNet using KD training Accuracy' )
ax[1].plot(mobilenetKdTesting[:,1], label = 'MobileNet using KD training Sensitivity' )
ax[1].plot(mobilenetKdTesting[:,2], label = 'MobileNet using KD training Specificity' )
ax[1].plot(mobilenetTesting[:,0], label = 'MobileNet training Accuracy' )
ax[1].plot(mobilenetTesting[:,1], label = 'MobileNet training Sensitivity' )
ax[1].plot(mobilenetTesting[:,2], label = 'MobileNet training Specificity' )
ax[1].set_xlabel('Epochs', fontsize = 16)
ax[1].set_ylabel('Percentage (%)', fontsize = 16)
ax[1].set_title(' Testing Performance Across Epochs', fontsize = 20)
ax[1].legend()
plt.show()
f.savefig('Figures/fig7.png', bbox_inches='tight')

# Temperature Vs Accuracy

In [ ]:
testPerformance = []
temperatureValues = [1, 2, 4, 16, 32, 64]
for temp in temperatureValues:
    st = getMobileNetModel()
    stPerf = train_and_evaluate_mobileNet_using_KD(st, restNet, Train_Data, Test_Data, Train_Label, Test_Label, 0.5, temp, 10, 0.001)
    stPerf = train_and_evaluate_mobileNet_using_KD(st, restNet, Train_Data, Test_Data, Train_Label, Test_Label, 0.5, temp, 10, 0.0001)
    testPerformance += [testModelPerformance(st,Test_Data,Test_Label)]
    del st 
    del stPerf
print(testPerformance)

In [ ]:
testPerformance =  np.asarray(testPerformance)
print(teacherPerformance.shape)
# f, ax = plt.subplots(figsize=(16, 4))

# ax.plot(tempratureValues,testACC )
# ax.set_xlabel('Temperature', fontsize = 16)
# ax.set_ylabel('Accuracy (%)', fontsize = 16)
# ax.set_title(' Testing Accuracy of Student Model trained using KD Accuracy', fontsize = 20)
# ax.legend()
# plt.show()
# f.savefig('Figures/fig8.png', bbox_inches='tight')

In [ ]:
# your code start from here for step 8
from keras_flops import get_flops
TeacherParameters = restNet.summary()
StudentParameters = mobileNetKD.summary()

flops = get_flops(restNet, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")

flops = get_flops(mobileNetKD, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")